## Laboratorio 5 - Modelación y Simulación
### Ejercicio 1

Stefano Aragoni, Carol Arévalo, Luis Santos

----------------

#### Ejercicio 1 - Depredador - Presa
Debe crear una simulación de depredador-presa utilizando modelos basados en agentes. Modele varios tipos de
depredadores con diferentes comportamientos y agregue obstáculos que afecten los movimientos de los agentes.

Requisitos:
- Cree una clase Predator con diferentes subclases para diferentes tipos de depredadores.
- Modele diferentes comportamientos para distintos tipos de depredadores, como depredadores "cazadores" que persiguen activamente a sus presas y depredadores "emboscadores" que esconden y sorprenden a sus presas.
- Agregue obstáculos que los agentes deben rodear.
- Simule los movimientos de los agentes y visualice sus interacciones

----------------

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import uuid

In [2]:
class Obstacle():
    def __init__(self, x, y):
        self.position = np.array([x, y])

In [ ]:
class Predator():
    def __init__(self, x, y):
        self.position = np.array([x, y])
        self.instance_hash = uuid.uuid4().hex

In [ ]:
class Hunter(Predator):
    def __init__(self, x, y):
        super().__init__(x, y)
    
    def update(self, prey_list, obstacle_list):
        # Regla 1: 
        pass

In [ ]:
class Ambusher(Predator):
    def __init__(self, x, y):
        super().__init__(x, y)
    
    def update(self, prey_list, obstacle_list):
        # Regla 1:
        pass

In [3]:
class Prey():
    def __init__(self, x, y):
        self.position = np.array([x, y])
        self.alive = True
        self.instance_hash = uuid.uuid4().hex
        self.speed = np.random.randint(1, 3)
    
    def update(self, predator_list, obstacle_list):

        # ------------------------------------------------------------
        # Regla 0: Que no se lo coma el depredador - Verificar si el depredador lo atrapó
        for predator in predator_list:
            if np.linalg.norm(new_position - predator.position) < 3:
                self.alive = False
                return

        # ------------------------------------------------------------
        # Regla 1: Escapar - Huir del depredador más cercano
        closest_predator = min(predator_list, key=lambda predator: np.linalg.norm(self.position - predator.position))

        if np.linalg.norm(self.position - closest_predator.position) < 10:
            # Si el depredador está cerca, huir de él
            direction = self.position - closest_predator.position           # Calcular la dirección hacia el depredador más cercano 

        else:
            # Si el depredador está lejos, moverse o no moverse aleatoriamente
            if np.random.random() < 0.5:
                direction = np.random.random(2) * 2 - 1                      # Moverse aleatoriamente
            else:
                direction = np.array([0, 0])

        # ------------------------------------------------------------
        # Regla 2: Evitar obstáculos - Evitar el obstáculo más cercano
        closest_obstacle = min(obstacle_list, key=lambda obstacle: np.linalg.norm(self.position - obstacle.position))

        if np.linalg.norm(self.position - closest_obstacle.position) < 5:
            # Si el obstáculo está cerca, alejarse de él
            obstacle_direction = self.position - closest_obstacle.position
            obstacle_direction = obstacle_direction / np.linalg.norm(obstacle_direction)

        else:
            obstacle_direction = np.array([0, 0])

        # ------------------------------------------------------------
        # Regla 3: Alejarse de depredadores y obstáculos
        direction = (direction + obstacle_direction) / 2
        new_position = self.position + (direction / np.linalg.norm(direction)) * self.speed

        # ------------------------------------------------------------
        # Regla 4: Que no se salga del mapa - Restringir la nueva posición dentro de los límites del mapa
        new_position = np.clip(new_position, 0, 100 - 1)

        self.position = new_position


In [ ]:
# Simulación
num_steps = 100
for step in range(num_steps):
    for predator in predators:
        for p in prey:
            predator.update(p)
    
    for p in prey:
        p.update(predators)

# Visualización (puedes adaptar esto según tus necesidades)
plt.figure(figsize=(8, 8))
for obstacle in obstacles:
    plt.plot(obstacle.position[0], obstacle.position[1], 'ro')

for predator in predators:
    plt.plot(predator.position[0], predator.position[1], 'bo')

for p in prey:
    plt.plot(p.position[0], p.position[1], 'go')

plt.xlim(0, map_size)
plt.ylim(0, map_size)
plt.show()